## Procesamiento inicial de los ficheros de audio

En esta parte lo único que voy a hacer va a ser transformar el fichero de auido a **.mp3**. Para todo lo que es el procesamiento de los audios, se usará la librería [pydub](https://pydub.com)

In [2]:
pip install --upgrade pip 

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
#Checkeo que se encuentra el archivo
os.path.exists("./audios/audio_prueba.ogg")

True

In [5]:
from pydub import AudioSegment
sound = AudioSegment.from_file("./audios/audio_prueba.ogg") #Puedes meter aquí cualquier tipo de audio

sound.export("./audios/salida/audio_prueba.mp3", format="mp3", bitrate="128k")

<_io.BufferedRandom name='./audios/salida/audio_prueba.mp3'>

## Uso del modelo



In [ ]:
pip install --upgrade transformers datasets[audio] accelerate

In [6]:
import torch
print("PyTorch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

PyTorch: 2.5.1+cu121
CUDA disponible: True
GPU: NVIDIA GeForce RTX 4060


In [5]:
import torch
import torchaudio
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor

# Configuración del dispositivo y tipo de dato
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Cargar modelo y procesador
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    use_safetensors=True
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

# === 1. Cargar el archivo de audio ===
audio_path = "./audios/salida/audio_prueba.mp3"
waveform, sampling_rate = torchaudio.load(audio_path)

# === 2. Convertir el audio a 16 kHz si es necesario ===
if sampling_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
    waveform = resampler(waveform)
    sampling_rate = 16000

# === 3. Preprocesar el audio ===
inputs = processor(
    waveform[0],
    sampling_rate=sampling_rate,
    return_tensors="pt"
)

# Mover los datos procesados al dispositivo (GPU/CPU) y tipo de dato
inputs = {k: v.to(device=device, dtype=torch_dtype) for k, v in inputs.items()}

# === 4. Crear un "prompt" con los nombres ===
personajes = "Zephir, Garl, Gorl, Iuman, Jeckiran, Tara"
prompt_ids = processor.tokenizer(personajes, return_tensors="pt").input_ids.to(device)

# Convertir los IDs en formato que acepta Whisper para forzar tokens al inicio
forced_decoder_ids = [(i + 1, token.item()) for i, token in enumerate(prompt_ids[0])]

# === 5. Generar la transcripción ===
generated_ids = model.generate(
    **inputs,
    forced_decoder_ids=forced_decoder_ids,
    max_new_tokens=448,
)

# === 6. Decodificar el texto generado ===
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Mostrar la transcripción
print("\n📄 Transcripción:")
print(transcription)


ValueError: The length of `decoder_input_ids`, including special start tokens, prompt tokens, and previous tokens, is 23,  and `max_new_tokens` is 448. Thus, the combined length of `decoder_input_ids` and `max_new_tokens` is: 471. This exceeds the `max_target_positions` of the Whisper model: 448. You should either reduce the length of your prompt, or reduce the value of `max_new_tokens`, so that their combined length is less than 448.

In [ ]:
print(transcription)